# Christelle


In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import os
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D, Conv2D

from imblearn.combine import SMOTETomek
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import ADASYN
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Reshape
from imblearn.over_sampling import SMOTE
from keras.layers import LSTM

In [11]:
import pandas as pd


malta_loc_18 = '../data/Malta-Budget-2018-dataset-v1.csv'
malta_loc_19 = '../data/Malta-Budget-2019-dataset-v1.csv'
malta_loc_20 = '../data/Malta-Budget-2020-dataset-v1.csv'

malta_data_18 = pd.read_csv(malta_loc_18)
malta_data_19 = pd.read_csv(malta_loc_19)
malta_data_20 = pd.read_csv(malta_loc_20)

print(malta_data_18.info())
print(malta_data_19.info())
print(malta_data_20.info())

malta_data_19 = malta_data_19.rename(columns={'Off-topic ':'Off-topic'})
combined_data = pd.concat([malta_data_18, malta_data_19, malta_data_20], ignore_index=True)
combined_data.info()

clean_data = combined_data.dropna(subset=['Online Post Text'])
clean_data = clean_data.drop(['Twitter ID', 'Related Online Post ID', 'Source ID','Off-topic'], axis=1)
clean_data = clean_data[clean_data['Language'] == 0] # get all data that is in english 
clean_data = clean_data.drop(['Language'], axis=1)
clean_data = clean_data.rename(columns={'Online Post ID':'ID','Online Post Text':'Text'})
clean_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Online Post ID          2142 non-null   int64  
 1   Twitter ID              1673 non-null   float64
 2   Related Online Post ID  333 non-null    float64
 3   Source ID               2142 non-null   object 
 4   Online Post Text        469 non-null    object 
 5   Subjectivity            2142 non-null   int64  
 6   Sentiment Polarity      2142 non-null   object 
 7   Emotion                 2142 non-null   object 
 8   Sarcasm                 2142 non-null   int64  
 9   Irony                   2142 non-null   int64  
 10  Negation                2142 non-null   int64  
 11  Off-topic               2142 non-null   int64  
 12  Language                2142 non-null   int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 217.7+ KB
None
<class 'pandas.core.frame.Dat

,ID,Text,Subjectivity,Sentiment Polarity,Emotion,Sarcasm,Irony,Negation
0,20180001,Great BUDGET . Even cigarettes were not touche...,1,positive,trust,0,0,1
1,20180002,I haven't exactly scanned the budget throughou...,1,negative,disgust,0,0,1
2,20180003,There’s already smoking cessation programs for...,1,neutral,anticipation,0,0,0
3,20180004,So should alcohol and fuel for private vehicle...,1,negative,sadness,0,0,0
4,20180005,Practical? You should say that in a third worl...,1,negative,anger,0,0,1


In [12]:
import string
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
from transformers import BertTokenizer

processed_data = clean_data.copy(deep=True)

def remove_special_characters(text):
    pattern = re.compile(r'[^a-zA-Z\s]')
    return pattern.sub('', text)

# Remove URLs and HTML tags
processed_data['Text'] = processed_data['Text'].str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
processed_data['Text'] = processed_data['Text'].str.replace(r'<.*?>', '', regex=True)

# Expand contractions
processed_data['Text'] = processed_data['Text'].apply(lambda x: contractions.fix(x))

# Convert to lowercase
processed_data['Text'] = processed_data['Text'].str.lower()

# Remove punctuation
processed_data['Text'] = processed_data['Text'].str.replace(f"[{string.punctuation}]", " ", regex=True)

# Remove numbers
processed_data['Text'] = processed_data['Text'].str.replace(r'\d+', '', regex=True)

# Remove special characters
processed_data['Text'] = processed_data['Text'].apply(remove_special_characters)

# Remove stop words
stop_words = set(stopwords.words('english'))
processed_data['Text'] = processed_data['Text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Remove extra whitespace
processed_data['Text'] = processed_data['Text'].str.strip()
processed_data['Text'] = processed_data['Text'].str.replace(r'\s+', ' ', regex=True)

# Lemmatize
lemmatizer = WordNetLemmatizer()
processed_data['Text'] = processed_data['Text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()))

# Tokenize
# tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment") #96574
# tokenizer_features = 110000
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #30522 
tokenizer_features = 30522
processed_data['tokens'] = processed_data['Text'].apply(lambda x: tokenizer.tokenize(x)) 

max_words = processed_data['Text'].apply(lambda x: len(x.split())).max()
print(f"The longest record is : {max_words} words")

max_tokens = processed_data['tokens'].apply(lambda x: len(x)).max()
print(f"The longest record is : {max_words} tokens")

def encode_texts(texts, tokenizer, max_len): 
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return input_ids, attention_masks

processed_data['padded'], processed_data['masks'] = encode_texts(processed_data['Text'].tolist(), tokenizer, 100)

# Encode
# processed_data['encoded'] = processed_data['tokens'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))

processed_data.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


The longest record is : 171 words
The longest record is : 171 tokens


,ID,Text,Subjectivity,Sentiment Polarity,Emotion,Sarcasm,Irony,Negation,tokens,padded,masks
0,20180001,great budget even cigarette touched great work...,1,positive,trust,0,0,1,"[great, budget, even, cigarette, touched, grea...","[101, 2307, 5166, 2130, 9907, 5028, 2307, 2147...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,20180002,exactly scanned budget throughout earth make i...,1,negative,disgust,0,0,1,"[exactly, scanned, budget, throughout, earth, ...","[101, 3599, 11728, 5166, 2802, 3011, 2191, 166...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,20180003,already smoking cessation program people want ...,1,neutral,anticipation,0,0,0,"[already, smoking, ce, ##ssa, ##tion, program,...","[101, 2525, 9422, 8292, 11488, 3508, 2565, 211...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,20180004,alcohol fuel private vehicle raising tax cigar...,1,negative,sadness,0,0,0,"[alcohol, fuel, private, vehicle, raising, tax...","[101, 6544, 4762, 2797, 4316, 6274, 4171, 9907...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,20180005,practical say third world country supposed eur...,1,negative,anger,0,0,1,"[practical, say, third, world, country, suppos...","[101, 6742, 2360, 2353, 2088, 2406, 4011, 2647...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


## Emotions Detection through Lexicon

In [13]:
# Load NRC lexicon
nrc_emotion_lexicon_path = '../Lexicons/NRC/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
nrc_emotion_lexicon = pd.read_csv(nrc_emotion_lexicon_path, sep='\t', header=None, names=['word', 'emotion', 'association'])


In [14]:
print(nrc_emotion_lexicon.head())
print(nrc_emotion_lexicon['word'].value_counts().head())

    word       emotion  association
0  aback         anger            0
1  aback  anticipation            0
2  aback       disgust            0
3  aback          fear            0
4  aback           joy            0
aback        10
polarity     10
pointedly    10
pointer      10
pointless    10
Name: word, dtype: int64


In [15]:
# Filter emotions of interest
emotions_of_interest = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']
nrc_emotion_lexicon = nrc_emotion_lexicon[nrc_emotion_lexicon['emotion'].isin(emotions_of_interest)]
nrc_emotion_lexicon = nrc_emotion_lexicon[nrc_emotion_lexicon['association'] == 1]  # Keep only rows where association is 1

In [16]:
# Convert the lexicon to a dictionary for faster lookup
emotion_dict = {}
for _, row in nrc_emotion_lexicon.iterrows():
    word = row['word']
    emotion = row['emotion']
    if word in emotion_dict:
        emotion_dict[word].append(emotion)
    else:
        emotion_dict[word] = [emotion]

print(emotion_dict)  # Optional: Print the dictionary to verify its structure


{'abacus': ['trust'], 'abandon': ['fear', 'sadness'], 'abandoned': ['anger', 'fear', 'sadness'], 'abandonment': ['anger', 'fear', 'sadness', 'surprise'], 'abbot': ['trust'], 'abduction': ['fear', 'sadness', 'surprise'], 'aberration': ['disgust'], 'abhor': ['anger', 'disgust', 'fear'], 'abhorrent': ['anger', 'disgust', 'fear'], 'abject': ['disgust'], 'abnormal': ['disgust'], 'abolish': ['anger'], 'abominable': ['disgust', 'fear'], 'abomination': ['anger', 'disgust', 'fear'], 'abortion': ['disgust', 'fear', 'sadness'], 'abortive': ['sadness'], 'abrupt': ['surprise'], 'abscess': ['sadness'], 'absence': ['fear', 'sadness'], 'absent': ['sadness'], 'absentee': ['sadness'], 'absolution': ['joy', 'trust'], 'abundance': ['anticipation', 'disgust', 'joy', 'trust'], 'abundant': ['joy'], 'abuse': ['anger', 'disgust', 'fear', 'sadness'], 'abysmal': ['sadness'], 'abyss': ['fear', 'sadness'], 'academic': ['trust'], 'accelerate': ['anticipation'], 'accident': ['fear', 'sadness', 'surprise'], 'accident

In [17]:
from nltk.tokenize import word_tokenize  # Optional: for better tokenization

def detect_emotions(text, emotion_dict, emotions_of_interest):
    tokens = word_tokenize(text.lower())  # Use nltk's tokenizer for better handling of punctuation
    emotions_count = {emotion: 0 for emotion in emotions_of_interest}
    
    for token in tokens:
        if token in emotion_dict:
            for emotion in emotion_dict[token]:
                emotions_count[emotion] += 1
                
    return emotions_count


In [18]:
# Detect emotions
processed_data['emotions'] = processed_data['Text'].apply(lambda x: detect_emotions(x, emotion_dict, emotions_of_interest))

# Convert emotions count to DataFrame
emotions_df = pd.json_normalize(processed_data['emotions'])

# Create new DataFrame with only 'Text' and the generated emotion scores
result_df = pd.concat([processed_data['Text'], emotions_df], axis=1)

# Save the new DataFrame as a CSV file
csv_file_path = 'text_with_emotion_scores.csv'
result_df.to_csv(csv_file_path, index=False)

print("CSV file saved successfully.")


CSV file saved successfully.


## Polarity Detection

# Pos vs Neg Scores

In [22]:
import pandas as pd

# Paths to the NRC sentiment lexicon files
positive_lexicon_path = '../Lexicons/NRC/OneFilePerEmotion/positive-NRC-Emotion-Lexicon.txt'
negative_lexicon_path = '../Lexicons/NRC/OneFilePerEmotion/negative-NRC-Emotion-Lexicon.txt'

# Load the positive and negative lexicons
positive_lexicon = pd.read_csv(positive_lexicon_path, sep='\t', header=None, names=['word', 'association'])
negative_lexicon = pd.read_csv(negative_lexicon_path, sep='\t', header=None, names=['word', 'association'])

# Extract words from the lexicons
positive_words = positive_lexicon[positive_lexicon['association'] == 1]['word'].tolist()
negative_words = negative_lexicon[negative_lexicon['association'] == 1]['word'].tolist()

# Function to calculate positive and negative polarity scores
def calculate_polarity_scores(text, positive_words, negative_words):
    words = text.split()
    pos_count = sum(1 for word in words if word in positive_words)
    neg_count = sum(1 for word in words if word in negative_words)
    total_sentiment_count = pos_count + neg_count
    
    # Calculate positive and negative scores
    pos_score = pos_count / (total_sentiment_count + 1e-6) 
    neg_score = neg_count / (total_sentiment_count + 1e-6)  
    
    return pos_score, neg_score

# Apply the polarity score calculation to your dataset
processed_data[['positive_score', 'negative_score']] = processed_data['Text'].apply(
    lambda x: pd.Series(calculate_polarity_scores(x, positive_words, negative_words))
)

# View the results
print(processed_data[['Text', 'positive_score', 'negative_score']].head())


                                                Text  positive_score  \
0  great budget even cigarette touched great work...        0.333333   
1  exactly scanned budget throughout earth make i...        0.000000   
2  already smoking cessation program people want ...        0.250000   
3  alcohol fuel private vehicle raising tax cigar...        0.000000   
4  practical say third world country supposed eur...        0.000000   

   negative_score  
0        0.666666  
1        1.000000  
2        0.750000  
3        1.000000  
4        0.000000  


# Pos vs Neg Count

In [24]:
import pandas as pd

# Paths to the NRC sentiment lexicon files
positive_lexicon_path = '../Lexicons/NRC/OneFilePerEmotion/positive-NRC-Emotion-Lexicon.txt'
negative_lexicon_path = '../Lexicons/NRC/OneFilePerEmotion/negative-NRC-Emotion-Lexicon.txt'

# Load the positive and negative lexicons
positive_lexicon = pd.read_csv(positive_lexicon_path, sep='\t', header=None, names=['word', 'association'])
negative_lexicon = pd.read_csv(negative_lexicon_path, sep='\t', header=None, names=['word', 'association'])

# Extract words from the lexicons
positive_words = positive_lexicon[positive_lexicon['association'] == 1]['word'].tolist()
negative_words = negative_lexicon[negative_lexicon['association'] == 1]['word'].tolist()

# Function to calculate raw positive and negative word counts
def count_polarity_words(text, positive_words, negative_words):
    words = text.split()
    pos_count = sum(1 for word in words if word in positive_words)
    neg_count = sum(1 for word in words if word in negative_words)
    
    return pos_count, neg_count

# Apply the count calculation to your dataset
processed_data[['positive_count', 'negative_count']] = processed_data['Text'].apply(
    lambda x: pd.Series(count_polarity_words(x, positive_words, negative_words))
)

# View the results
print(processed_data[['Text', 'positive_count', 'negative_count']].head())


                                                Text  positive_count  \
0  great budget even cigarette touched great work...               1   
1  exactly scanned budget throughout earth make i...               0   
2  already smoking cessation program people want ...               1   
3  alcohol fuel private vehicle raising tax cigar...               0   
4  practical say third world country supposed eur...               0   

   negative_count  
0               2  
1               3  
2               3  
3               6  
4               0  
